In [1]:
!pip install efficient-apriori

In [0]:
from efficient_apriori import apriori

In [3]:
!git clone https://github.com/AndreaJJCC/CategorySuggestion.git
# Other necessary installations/downloads
import nltk
nltk.download('punkt')

Cloning into 'CategorySuggestion'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 3), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (15/15), done.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# Imports
import os
import json
import pandas as pd
import re 
import numpy as np

In [5]:
# Define file paths
main_dir = '/content/CategorySuggestion/'
business_dir = main_dir + 'yelp_academic_dataset_business.json'
os.chdir('/content/CategorySuggestion/')
!unzip -o /content/CategorySuggestion/yelp_academic_dataset_business.zip

Archive:  /content/CategorySuggestion/yelp_academic_dataset_business.zip
  inflating: yelp_academic_dataset_business.json  


In [0]:
# Define function to load files
# Returns dictionary of json objects
def load_data( directory):
  with open(directory) as f:
    data = []
    for line in f:
      data.append(json.loads(line))
  return data

In [7]:
# Load json files as dictionaries
# and convert dictionaries to pandas dataframe
business_df = pd.DataFrame.from_dict(load_data(business_dir))
business_records = business_df.shape[0]
print('Business data\n' + 'Loaded ' + str(business_records) + ' records.')
business_df.head(2) 

Business data
Loaded 188593 records.


,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV


In [0]:
# Define function to convert text to lowercase and  remove punctuation
def to_lower_and_punc( col ):
  return col.astype(str)\
            .str.lower()\
            .str.replace('[^a-z\s-]', '')

In [0]:
# Convert categories column to lowercase
items = to_lower_and_punc(business_df.categories).apply(nltk.word_tokenize)


In [0]:
# Create list of tuples (categories) to input to apriori algorithm
items_list = []
for item in items:
  items_list.append(tuple(item))

In [0]:
# Input: list[tuples], min_support float, min_confidence float-max = 1, max_length int
# output: dict{ numOfValuesInSets:itemsets}, list[rules]
itemsets, rules = apriori(items_list, min_support=0.003,  min_confidence=1, max_length = 5)

**Support:** popularity of an item. = number of transactions containing item i/ total number of transactions

**Confidence:** the likelihood that an item B is also bought if item A is bought. = the number of transactions where A and B are bought together/ total number of transactions where A is bought

**Lift:** the likelihood of buying a A and B together is x times more than the likelihood of just buying B. = Confidence( A -> B)/Support(B). 1 => no association, < 1 => unlikely to be bought together, > 1 => more likely to ge bought together.


In [12]:
# Create a list of sets from dict{ numOfValuesInSets:{itemsets}}
# i.e Go from {1: {('accessories',): 44, ('yogurt',): 145},  <-set size = 1
#              2: {('accessories', 'computer'): 29}}  <-set size = 2
# to [{'accessories'}, {'yogurt'}, {'accessories', 'computer'}]
sets_list = []
for value in itemsets.values():
  for element in value:
    sets_list.append(element)
print(sets_list[0:5])

[('accessories',), ('active',), ('acupuncture',), ('adult',), ('agents',)]


In [0]:
#mylist = [{'hello'}, {'this', 'is'}, {'a'}, {'list', 'of', 'lists'}]
#newlist = []
with open(main_dir + 'sets.json', 'w') as outfile:
 # for item in mylist:
 #   newlist.append(list(item))
    
  json.dump(sets_list, outfile)

In [14]:
"""
with open(main_dir + 'sets.json', 'r') as f:
  r = json.load(f)
  for item in r:
    print(set(item))
"""

"\nwith open(main_dir + 'sets.json', 'r') as f:\n  r = json.load(f)\n  for item in r:\n    print(set(item))\n"